Reference - https://github.com/lijichang/CVPR2021-SSDA.git

# Data Loading

In [ ]:
!wget -P /content/drive/MyDrive/CVPR/clipart http://csr.bu.edu/ftp/visda/2019/multi-source/clipart.zip  
!wget -P /content/drive/MyDrive/CVPR/sketch http://csr.bu.edu/ftp/visda/2019/multi-source/sketch.zip 

--2022-05-05 13:22:27--  http://csr.bu.edu/ftp/visda/2019/multi-source/clipart.zip
Resolving csr.bu.edu (csr.bu.edu)... 128.197.11.70
Connecting to csr.bu.edu (csr.bu.edu)|128.197.11.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1272750339 (1.2G) [application/zip]
Saving to: ‘/content/drive/MyDrive/CVPR/clipart/clipart.zip’

clipart.zip         100%[===================>]   1.18G  32.8MB/s    in 38s     

2022-05-05 13:23:05 (31.8 MB/s) - ‘/content/drive/MyDrive/CVPR/clipart/clipart.zip’ saved [1272750339/1272750339]

--2022-05-05 13:23:05--  http://csr.bu.edu/ftp/visda/2019/multi-source/sketch.zip
Resolving csr.bu.edu (csr.bu.edu)... 128.197.11.70
Connecting to csr.bu.edu (csr.bu.edu)|128.197.11.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2633992926 (2.5G) [application/zip]
Saving to: ‘/content/drive/MyDrive/CVPR/sketch/sketch.zip’

sketch.zip          100%[===================>]   2.45G  32.5MB/s    in 79s     

2022-05-05 1

In [ ]:
!unzip /content/drive/MyDrive/CVPR/sketch/sketch.zip -d /content/drive/MyDrive/CVPR/sketch

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/CVPR/sketch/sketch/tree/sketch_323_000134.jpg  
  inflating: /content/drive/MyDrive/CVPR/sketch/sketch/tree/sketch_323_000135.jpg  
  inflating: /content/drive/MyDrive/CVPR/sketch/sketch/tree/sketch_323_000136.jpg  
  inflating: /content/drive/MyDrive/CVPR/sketch/sketch/tree/sketch_323_000137.jpg  
  inflating: /content/drive/MyDrive/CVPR/sketch/sketch/tree/sketch_323_000138.jpg  
  inflating: /content/drive/MyDrive/CVPR/sketch/sketch/tree/sketch_323_000139.jpg  
  inflating: /content/drive/MyDrive/CVPR/sketch/sketch/tree/sketch_323_000140.jpg  
  inflating: /content/drive/MyDrive/CVPR/sketch/sketch/tree/sketch_323_000141.jpg  
  inflating: /content/drive/MyDrive/CVPR/sketch/sketch/tree/sketch_323_000142.jpg  
  inflating: /content/drive/MyDrive/CVPR/sketch/sketch/tree/sketch_323_000143.jpg  
  inflating: /content/drive/MyDrive/CVPR/sketch/sketch/tree/sketch_323_000144.jpg  
  inflating: /content/dri

In [ ]:
!unzip /content/drive/MyDrive/CVPR/clipart/clipart.zip -d /content/drive/MyDrive/CVPR/clipart

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/CVPR/clipart/clipart/train/trunk14/cda18b86d1c7d7a41ced784338090ff3.jpg  
  inflating: /content/drive/MyDrive/CVPR/clipart/clipart/train/trunk14/27d8d80c503afc1bc5e66c2e5035ef68.jpg  
  inflating: /content/drive/MyDrive/CVPR/clipart/clipart/train/trunk14/d07a8a0a1f94df9553cc61184251a8c2.jpg  
  inflating: /content/drive/MyDrive/CVPR/clipart/clipart/train/trunk14/c284e860af9eac89a084b37d6077ff01.jpg  
  inflating: /content/drive/MyDrive/CVPR/clipart/clipart/train/trunk14/059435c45d88db51ccd4ab3bc3b655e4.jpg  
  inflating: /content/drive/MyDrive/CVPR/clipart/clipart/train/trunk14/271e061eacf95e3e1eea41a4e567b3a7.jpg  
  inflating: /content/drive/MyDrive/CVPR/clipart/clipart/train/trunk14/4038813b39c59388a0a0c03a68675f09.jpg  
  inflating: /content/drive/MyDrive/CVPR/clipart/clipart/train/trunk14/e987d5068a4d1d29ee30f0b9ef57a8a8.jpg  
  inflating: /content/drive/MyDrive/CVPR/clipart/clipart/train/trunk1

# Training on VISDA17

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

Mounted at /content/drive


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/CVPR')
sys.path.insert(0,'/content/drive/MyDrive/CVPR/model')

In [ ]:
from __future__ import print_function
import argparse
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from model.resnet import resnet34
from model.basenet import AlexNetBase, Predictor
from utils.utils import weights_init
from utils.lr_schedule import inv_lr_scheduler
from utils.return_dataset import return_dataset
from losses import BCE_softlabels, sigmoid_rampup, get_losses_unlabeled
import time

In [ ]:
class Arguments():

  def __init__(self):
        self.steps = 5000                     #max number of iterations
        self.method = 'CDAC'                   #mico or mist
        self.lr = 0.01                         #learning rate
        self.lr_f = 1.0
        self.multi = 0.1                       #lr multiplication factor
        self.T = 0.05
        self.lamda = 0.1
        self.save_check = True                #save checkpoint or not
        self.checkpath ='/content/drive/MyDrive/CVPR/checkpoints'         #dir to save checkpoint
        self.seed = 1                          #random seed (default:1)
        self.log_interval = 100                #how many batches to wait before logging training status
        self.save_interval = 500               #how many batches to wait before saving a model
        self.net = 'resnet34'                  #which network to use
        self.source = 'synthetic'                   #source domain
        self.target = 'real'                #target domain
        self.st = 0
        self.dataset = 'VISDA17'
        self.num = 3                           #3-shot/1-shot 
        self.rampup_length = 2000
        self.rampup_coef = 30.0                          #threshold
        self.root = '/content/drive/MyDrive/CVPR'
        self.topk = 5
        self.threshold = 0.95
        self.remark = ''
        self.eval = False

In [ ]:
args = Arguments()
torch.cuda.manual_seed(args.seed)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print('Dataset %s Source %s Target %s Labeled num perclass %s Network %s' % (
args.dataset, args.source, args.target, args.num, args.net))
source_dataset, target_dataset, target_dataset_unl, target_dataset_val, target_dataset_test, class_list = return_dataset(
    args)

Dataset VISDA17 Source synthetic Target real Labeled num perclass 3 Network resnet34
12 classes in this dataset


In [ ]:
def folder_preparation(args):
    import datetime
    nowtime = datetime.datetime.now().strftime('%m%d%H%M%S')

    main_path = 'record/%s_%s_%s_net_%s_%s_to_%s_num_%s_%s' % (nowtime, args.dataset, args.method, args.net, args.source,
                              args.target, args.num, args.remark)
    # logs saving
    logs_file = os.path.join(main_path, 'logs')
                              
    # checkpath saving
    checkpath = os.path.join(main_path, 'checkpath')
    
    if not os.path.exists(main_path):
        os.makedirs(main_path)
        
    if not os.path.exists(checkpath):
        os.makedirs(checkpath)

    return main_path, logs_file, checkpath


main_path, logs_file, checkpath = folder_preparation(args)
print("Main path to save: {}".format(main_path))

Main path to save: record/0506101949_VISDA17_CDAC_net_resnet34_synthetic_to_real_num_3_


In [ ]:
if args.net == 'resnet34':
    G = resnet34()
    inc = 512
    bs = 16
elif args.net == "alexnet":
    G = AlexNetBase()
    inc = 4096
    bs = 32
else:
    raise ValueError('Model cannot be recognized.')

params = []
for key, value in dict(G.named_parameters()).items():
    if value.requires_grad:
        if 'classifier' not in key:
            params += [{'params': [value], 'lr': args.multi,
                        'weight_decay': 0.0005}]
        else:
            params += [{'params': [value], 'lr': args.multi * 10,
                        'weight_decay': 0.0005}]

F1 = Predictor(num_class=len(class_list), inc=inc, temp=args.T)
weights_init(F1)



G = nn.DataParallel(G)
F1 = nn.DataParallel(F1)
G = G.to(device)
F1 = F1.to(device)

opt = {}
opt["logs_file"] = logs_file
opt["checkpath"] = checkpath
opt["class_list"] = class_list

lr = args.lr

source_loader = torch.utils.data.DataLoader(source_dataset, batch_size=bs, num_workers=3, shuffle=True, drop_last=True)
target_loader = torch.utils.data.DataLoader(target_dataset, batch_size=min(bs, len(target_dataset)), num_workers=3,
                                            shuffle=True, drop_last=True)
target_loader_unl = torch.utils.data.DataLoader(target_dataset_unl, batch_size=bs * 2, num_workers=3, shuffle=True,
                                                drop_last=True)
target_loader_val = torch.utils.data.DataLoader(target_dataset_val, batch_size=min(bs, len(target_dataset_val)),
                                                num_workers=3, shuffle=True, drop_last=True)
target_loader_test = torch.utils.data.DataLoader(target_dataset_test, batch_size=bs * 2, num_workers=3, shuffle=True,
                                                 drop_last=True)



Downloading: "https://s3.amazonaws.com/pytorch/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
def train(device, opt):
    G.train()
    F1.train()

    optimizer_g = optim.SGD(params, momentum=0.9, weight_decay=0.0005, nesterov=True)
    optimizer_f = optim.SGD(list(F1.parameters()), lr=args.lr_f, momentum=0.9, weight_decay=0.0005, nesterov=True)

    def zero_grad_all():
        optimizer_g.zero_grad()
        optimizer_f.zero_grad()

    param_lr_g = []
    for param_group in optimizer_g.param_groups:
        param_lr_g.append(param_group["lr"])
    param_lr_f = []
    for param_group in optimizer_f.param_groups:
        param_lr_f.append(param_group["lr"])

    all_step = args.steps
    data_iter_s = iter(source_loader)
    data_iter_t = iter(target_loader)
    data_iter_t_unl = iter(target_loader_unl)
    len_train_source = len(source_loader)
    len_train_target = len(target_loader)
    len_train_target_semi = len(target_loader_unl)
    print(len_train_target_semi)

    best_acc = 0

    BCE = BCE_softlabels().to(device)
    criterion = nn.CrossEntropyLoss().to(device)

    start_time = time.time()
    for step in range(all_step):

        rampup = sigmoid_rampup(step, args.rampup_length)
        w_cons = args.rampup_coef * rampup

        optimizer_g = inv_lr_scheduler(param_lr_g, optimizer_g, step,
                                       init_lr=args.lr)
        optimizer_f = inv_lr_scheduler(param_lr_f, optimizer_f, step,
                                       init_lr=args.lr)
        lr_f = optimizer_f.param_groups[0]['lr']
        lr_g = optimizer_g.param_groups[0]['lr']

        if step % len_train_target == 0:
            data_iter_t = iter(target_loader)
        if step % len_train_target_semi == 0:
            data_iter_t_unl = iter(target_loader_unl)
        if step % len_train_source == 0:
            data_iter_s = iter(source_loader)
        data_t = next(data_iter_t)
        data_t_unl = next(data_iter_t_unl)
        data_s = next(data_iter_s)

        # load labeled source data
        x_s, target_s = data_s[0], data_s[1]
        im_data_s = x_s.to(device)
        gt_labels_s = target_s.to(device)

        # load labeled target data
        x_t, target_t = data_t[0], data_t[1]
        im_data_t = x_t.to(device)
        gt_labels_t = target_t.to(device)

        # load unlabeled target data
        x_tu, x_bar_tu, x_bar2_tu = data_t_unl[0], data_t_unl[3], data_t_unl[4]
        im_data_tu = x_tu.to(device)
        im_data_bar_tu = x_bar_tu.to(device)
        im_data_bar2_tu = x_bar2_tu.to(device)

        zero_grad_all()
        # construct losses for overall labeled data
        data = torch.cat((im_data_s, im_data_t), 0)
        target = torch.cat((gt_labels_s, gt_labels_t), 0)
        output = G(data)  # [batchsize, num_classes]
        out1 = F1(output)  # [batchsize, ]
        ce_loss = criterion(out1, target)

        ce_loss.backward(retain_graph=True)
        optimizer_g.step()
        optimizer_f.step()
        zero_grad_all()

        # construct losses for unlabeled target data
        aac_loss, pl_loss, con_loss = get_losses_unlabeled(args, G, F1, im_data=im_data_tu, im_data_bar=im_data_bar_tu,
                                                           im_data_bar2=im_data_bar2_tu, target=None, BCE=BCE,
                                                           w_cons=w_cons, device=device)
        loss = aac_loss + pl_loss + con_loss

        loss.backward()
        optimizer_g.step()
        optimizer_f.step()

        G.zero_grad()
        F1.zero_grad()
        zero_grad_all()

        if step % args.log_interval == 0:
            log_train = 'S {} T {} Train Ep: {} lr_f{:.6f} lr_g{:.6f}\n'.format(args.source, args.target,
                                                                                step, lr_f, lr_g)
            print(log_train)
            with open(opt["logs_file"], 'a') as f:
                f.write(log_train)

        if (step % args.save_interval) == 0 and step > 0 or (step == all_step - 1):
            loss_test, acc_test = test(target_loader_test)
            loss_val, acc_val = test(target_loader_val)
            G.train()
            F1.train()
            if acc_val >= best_acc:
                best_acc = acc_val
                best_acc_test = acc_test

            cur_time = time.time() - start_time
            print('Current acc test %f best acc test %f best acc val %f time cost %f sec.' % (
            acc_test, best_acc_test, acc_val, cur_time))

            with open(opt["logs_file"], 'a') as f:
                f.write('step %d current %f best %f val %f time cost %f sec.\n\n' % (
                step, acc_test, best_acc_test, acc_val, cur_time))

            G.train()
            F1.train()
            if args.save_check:
                print('Saving model')
                torch.save(G.state_dict(), os.path.join(opt["checkpath"],
                                                        "G_iter_model_{}_to_{}_step_{}.pth.tar".format(args.source,
                                                                                                       args.target,
                                                                                                       step)))
                torch.save(F1.state_dict(), os.path.join(opt["checkpath"],
                                                         "F1_iter_model_{}_to_{}_step_{}.pth.tar".format(args.source,
                                                                                                         args.target,
                                                                                                         step)))
            start_time = time.time()


def test(loader):
    G.eval()
    F1.eval()
    test_loss = 0
    correct = 0
    size = 0
    num_class = len(class_list)
    output_all = np.zeros((0, num_class))
    criterion = nn.CrossEntropyLoss().to(device)
    confusion_matrix = torch.zeros(num_class, num_class)
    with torch.no_grad():
        for batch_idx, data_t in enumerate(loader):
            im_data_t = data_t[0].to(device)
            gt_labels_t = data_t[1].to(device)
            feat = G(im_data_t)
            output1 = F1(feat)
            output_all = np.r_[output_all, output1.data.cpu().numpy()]
            size += im_data_t.size(0)
            pred1 = output1.data.max(1)[1]
            for t, p in zip(gt_labels_t.view(-1), pred1.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
            correct += pred1.eq(gt_labels_t.data).cpu().sum()
            test_loss += criterion(output1, gt_labels_t) / len(loader)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} F1 ({:.0f}%)\n'.format(test_loss, correct, size,
                                                                                    100. * correct / size))
    return test_loss.data, 100. * float(correct) / size


In [ ]:
train(device=device, opt=opt)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


S synthetic T real Train Ep: 0 lr_f0.010000 lr_g0.001000

S synthetic T real Train Ep: 100 lr_f0.009926 lr_g0.000993

S synthetic T real Train Ep: 200 lr_f0.009853 lr_g0.000985

S synthetic T real Train Ep: 300 lr_f0.009781 lr_g0.000978

S synthetic T real Train Ep: 400 lr_f0.009710 lr_g0.000971

S synthetic T real Train Ep: 500 lr_f0.009641 lr_g0.000964


Test set: Average loss: 1.2262, Accuracy: 24/32 F1 (75%)


Test set: Average loss: 0.7012, Accuracy: 27/32 F1 (84%)

Current acc test 75.000000 best acc test 75.000000 best acc val 84.375000 time cost 773.229804 sec.
Saving model
S synthetic T real Train Ep: 600 lr_f0.009572 lr_g0.000957

S synthetic T real Train Ep: 700 lr_f0.009505 lr_g0.000951

S synthetic T real Train Ep: 800 lr_f0.009439 lr_g0.000944

S synthetic T real Train Ep: 900 lr_f0.009374 lr_g0.000937

S synthetic T real Train Ep: 1000 lr_f0.009310 lr_g0.000931


Test set: Average loss: 1.3202, Accuracy: 26/32 F1 (81%)


Test set: Average loss: 0.7004, Accuracy: 28/32 F1

# Clipart to sketch

In [ ]:
class Arguments():

  def __init__(self):
        self.steps = 5000                     #max number of iterations
        self.method = 'CDAC'                   #mico or mist
        self.lr = 0.01                         #learning rate
        self.lr_f = 1.0
        self.multi = 0.1                       #lr multiplication factor
        self.T = 0.05
        self.lamda = 0.1
        self.save_check = True                #save checkpoint or not
        self.checkpath ='/content/drive/MyDrive/CVPR/checkpoints'         #dir to save checkpoint
        self.seed = 1                          #random seed (default:1)
        self.log_interval = 100                #how many batches to wait before logging training status
        self.save_interval = 500               #how many batches to wait before saving a model
        self.net = 'resnet34'                  #which network to use
        self.source = 'clipart'                   #source domain
        self.target = 'sketch'                #target domain
        self.st = 0
        self.dataset = 'multi'
        self.num = 3                           #3-shot/1-shot 
        self.rampup_length = 2000
        self.rampup_coef = 30.0                          #threshold
        self.root = '/content/drive/MyDrive/CVPR'
        self.topk = 5
        self.threshold = 0.95
        self.remark = ''
        self.eval = False

In [ ]:
args = Arguments()
torch.cuda.manual_seed(args.seed)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print('Dataset %s Source %s Target %s Labeled num perclass %s Network %s' % (
args.dataset, args.source, args.target, args.num, args.net))
source_dataset, target_dataset, target_dataset_unl, target_dataset_val, target_dataset_test, class_list = return_dataset(
    args)

Dataset multi Source clipart Target sketch Labeled num perclass 3 Network resnet34
126 classes in this dataset


In [ ]:
def folder_preparation(args):
    import datetime
    nowtime = datetime.datetime.now().strftime('%m%d%H%M%S')

    main_path = 'record/%s_%s_%s_net_%s_%s_to_%s_num_%s_%s' % (nowtime, args.dataset, args.method, args.net, args.source,
                              args.target, args.num, args.remark)
    # logs saving
    logs_file = os.path.join(main_path, 'logs')
                              
    # checkpath saving
    checkpath = os.path.join(main_path, 'checkpath')
    
    if not os.path.exists(main_path):
        os.makedirs(main_path)
        
    if not os.path.exists(checkpath):
        os.makedirs(checkpath)

    return main_path, logs_file, checkpath


main_path, logs_file, checkpath = folder_preparation(args)
print("Main path to save: {}".format(main_path))

Main path to save: record/0506153149_multi_CDAC_net_resnet34_clipart_to_sketch_num_3_


In [ ]:
if args.net == 'resnet34':
    G = resnet34()
    inc = 512
    bs = 16
elif args.net == "alexnet":
    G = AlexNetBase()
    inc = 4096
    bs = 32
else:
    raise ValueError('Model cannot be recognized.')

params = []
for key, value in dict(G.named_parameters()).items():
    if value.requires_grad:
        if 'classifier' not in key:
            params += [{'params': [value], 'lr': args.multi,
                        'weight_decay': 0.0005}]
        else:
            params += [{'params': [value], 'lr': args.multi * 10,
                        'weight_decay': 0.0005}]

F1 = Predictor(num_class=len(class_list), inc=inc, temp=args.T)
weights_init(F1)



G = nn.DataParallel(G)
F1 = nn.DataParallel(F1)
G = G.to(device)
F1 = F1.to(device)

opt = {}
opt["logs_file"] = logs_file
opt["checkpath"] = checkpath
opt["class_list"] = class_list

lr = args.lr

source_loader = torch.utils.data.DataLoader(source_dataset, batch_size=bs, num_workers=3, shuffle=True, drop_last=True)
target_loader = torch.utils.data.DataLoader(target_dataset, batch_size=min(bs, len(target_dataset)), num_workers=3,
                                            shuffle=True, drop_last=True)
target_loader_unl = torch.utils.data.DataLoader(target_dataset_unl, batch_size=bs * 2, num_workers=3, shuffle=True,
                                                drop_last=True)
target_loader_val = torch.utils.data.DataLoader(target_dataset_val, batch_size=min(bs, len(target_dataset_val)),
                                                num_workers=3, shuffle=True, drop_last=True)
target_loader_test = torch.utils.data.DataLoader(target_dataset_test, batch_size=bs * 2, num_workers=3, shuffle=True,
                                                 drop_last=True)



Downloading: "https://s3.amazonaws.com/pytorch/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
train(device=device, opt=opt)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


744


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


S clipart T sketch Train Ep: 0 lr_f0.010000 lr_g0.001000

S clipart T sketch Train Ep: 100 lr_f0.009926 lr_g0.000993

S clipart T sketch Train Ep: 200 lr_f0.009853 lr_g0.000985

S clipart T sketch Train Ep: 300 lr_f0.009781 lr_g0.000978

S clipart T sketch Train Ep: 400 lr_f0.009710 lr_g0.000971

S clipart T sketch Train Ep: 500 lr_f0.009641 lr_g0.000964


Test set: Average loss: 2.1395, Accuracy: 11814/23808 F1 (50%)


Test set: Average loss: 1.9447, Accuracy: 194/368 F1 (53%)

Current acc test 49.621976 best acc test 49.621976 best acc val 52.717391 time cost 2737.437171 sec.
Saving model
S clipart T sketch Train Ep: 600 lr_f0.009572 lr_g0.000957

S clipart T sketch Train Ep: 700 lr_f0.009505 lr_g0.000951

S clipart T sketch Train Ep: 800 lr_f0.009439 lr_g0.000944

S clipart T sketch Train Ep: 900 lr_f0.009374 lr_g0.000937

S clipart T sketch Train Ep: 1000 lr_f0.009310 lr_g0.000931


Test set: Average loss: 1.9142, Accuracy: 13119/23808 F1 (55%)


Test set: Average loss: 1.7163, Acc